# Numerical Methods 1
### [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman), [Matthew Piggott](http://www.imperial.ac.uk/people/m.d.piggott), [Christian Jacobs](http://www.christianjacobs.uk)

# Lecture ?: Numerical Linear Algebra III

## Learning objectives:

* Ill-conditioned matrices
* Direct vs iterative/indirect methods
* An example iternative algorithm

## Ill-conditioned matrices



## Direct vs iterative methods

Two types/families of methods exist to solve matrix systems.  These are termed *direct* methods and *iterative* (or *indirect*) methods.

Direct methods perform operations on the linear equations (the matrix system), e.g. the substitution of one eqution into another which we performed last week for your example $2\times 2$ system considered in MM1. This (and the subsequent Gaussian elimination algorithm) transformed the equations making up the linear system into equivalent ones with the aim of eliminating unknowns from some of the equations and hence allowing for easy solution through back (or forward) substitution.

In MM1 you learnt Cramer's rule which gives an explicit formula for the inverse of a matrix, or for the solution of a linear matrix system.  It was pointed out that the computational cost (in terms of arithmetic operations required; also termed complexity) scaled like $(n+1)!$, whereas the Gaussian elimination (which is basically the susbtitution method done above) scaled like $n^3$.  For large $n$ Gaussian elimination will clearly be more efficient - you considered the case where $n=100$ in MM1 for example. $n$ here refers to the number of unknowns or equations, or sometimes termed the *degrees of freedom* of the problem.

However, as pointed out above $n$ could be billions for hard-core applications such as in weather forecasting. In this case the $n^3$ operations required of a direct algorithm such as Gaussian elimination is also prohibitive. 

In order to reduce this cost, ideally to a level that is (close to) linear in $n$, *iterative* algorithms were devised. 

These start with a guess at the solution ($\pmb{x}_0$), they calculate the residual vector ($A\pmb{x}_0 - \pmb{b}$), and its norm (a scalar measure of a vector's size - e.g. the vector *2-norm* is just the square root of the sum of the squares of the components) which will obviously not be zero unless you were very lucky with your initial guess, and then *iteratively* seek to improve on this solution to drive down this residual norm.  This iteration will stop at some small (non-zero) residual norm tolerance level, yielding an approximation to the solution, but not the exact solution we would obtain with direct methods.  The residual norm tolerance stopping criteria therefore needs to be thought about carefully, e.g. depending on how accurate a solution $\pmb{x}$ we require.

Last week we considered Gaussian elimination (and back substitution) as examples of direct solution methods. We'll look briefly as iterative methods this week.